## <center id="c1"><h2> 🦜🔨 `LLM-Ops` - вскрываем чёрный ящик! </h2>

<img src="../images/bb.png" align="right" width="400" height="300" style="border-radius: 0.75rem;" alt="LangChain Image" />

### Оглавление ноутбука:

 * <a href="#c1"> 🧾 Debug от LangChain - самый простой!  </a>
 * <a href="#c2"> 🛠🦜 LangSmith и трейсы! </a> 
 * <a href="#look1"> 🔃🔪 Разделяем трейсы на проекты! </a>
 * <a href="#check1"> 🎭 Кастомизация трейсов! </a>
 * <a href="#6">🧸 Выводы и заключения</a>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
В этом уроке рассмотрим некоторые подходы и инструменты необходимые для отладки проектируемых AI-сервисов с LLM.

Отправляя запрос в сложную цепочку или агента, мы видим только итоговую генерацию, не понимая какие инструменты вызывались, какие звенья цепочки отработали и почему. Шлифуя какой-либо промпт под задачу тоже полезно отслеживать, как меняется поведение всей AI-системы.


# <center> 🔑 Вводим ключ 

In [1]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install langchain langchain-classic langchain-community langchain-openai openai wikipedia -q

In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [3]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

# course_api_key= "Введите API-ключ полученный в боте"
course_api_key = getpass(prompt='Введите API-ключ полученный в боте:')

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://api.vsellm.ru/")

Введите API-ключ полученный в боте: ········


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Допустим у нас есть база отзывов покупателей, мы хотим подать отзыв на вход модели, а на выходе получить готовый Python словарь(как представлено выше) для дальнейшего использования.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Давайте определим то, как мы хотим, чтобы выглядели выходные данные из LLM:

# <center id="c2"> 🧾 Debug от `LangChain` - самый простой!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Как мы уже вскольз упоминали в уроке про агентов, в `LangChain` есть встроенный Debug-режим.

Активируется строчкой кода `langchain.debug = True`<br>
    

In [8]:
from langchain_classic.agents import AgentExecutor, load_tools, create_react_agent
from langchain_classic import hub
import langchain 


tools = load_tools(["llm-math", "wikipedia"], llm=llm)

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "Сколько голов Месси и Криштиану Роналду забили в сумме?"})

{'input': 'Сколько голов Месси и Криштиану Роналду забили в сумме?',
 'output': 'Более 1650 голов в сумме забили Лионель Месси и Криштиану Роналду на октябрь 2023 года.'}

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Видим только запрос - ответ!

Добавим `verbose`. 

In [6]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Сколько голов Месси и Криштиану Роналду забили в сумме?"})



> Entering new AgentExecutor chain...
Чтобы ответить на вопрос о количестве голов Месси и Криштиану Роналду, мне нужно найти актуальную информацию о количестве голов, которые они забили в карьере на данный момент. Я буду использовать Википедию для поиска этой информации.

Action: wikipedia  
Action Input: "Месси количество голов 2023"  No good Wikipedia Search Result was foundПохоже, что я не смог найти подходящую информацию на Википедии о количестве голов Месси. Я попробую найти информацию о головах Роналду.

Action: wikipedia  
Action Input: "Роналду количество голов 2023"  No good Wikipedia Search Result was foundПохоже, что я не нашел актуальную информацию о количестве голов Месси и Роналду на Википедии. Вместо этого, мне нужно проверить, есть ли обновленные данные о карьере обоих игроков где-то еще. Я могу использовать доступные источники, такие как спортивные новости. 

Я попробую сначала найти общее количество голов Месси, а затем Роналду с помощью Википедии, если данные измен

{'input': 'Сколько голов Месси и Криштиану Роналду забили в сумме?',
 'output': '1812 голов.'}

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Видим больше информации, но нет конкретных запросов к модели, какие были `Messagess`, сколько токенов ушло.

Включаем `debug`!

In [7]:
langchain.debug = True

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "Сколько голов Месси и Криштиану Роналду забили в сумме?"})

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Сколько голов Месси и Криштиану Роналду забили в сумме?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run w

{'input': 'Сколько голов Месси и Криштиану Роналду забили в сумме?',
 'output': '1812'}

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Здесь уже наблюдаем все запросы, расход токенов на каждый, постепенное наполнение промптов промежуточной историей.

In [10]:
# Отключаем
langchain.debug = False

<div class="alert alert-warning" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**НО!** Даже на один запрос, и для простого агента с 2-мя инструментами, копаться в таких логах и искать нужную информацию уже не очень удобно. При увеличении системы таким способом отладки вообще будет невозможно пользоваться. 

# <center id="c2"> 🛠🦜 `LangSmith` и трейсы!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

`LangSmith` ([дока](https://docs.smith.langchain.com)) - это многофункциональная онлайн платформа от создателей `LangChain` для мониторинга, оценки качества и отладки приложений с LLM.

Основные сущности системы мониторинга в `LangSmith`:
* `Trace` - последовательность шагов, которые выполняет ваше приложение для перехода от ввода к выводу (по сути, один `invoke`).
* `Run` - каждый из этих отдельных шагов представлен выполнением (например, один запрос к LLM).
* `Project` - это просто набор трэйсов.

На картинке показаны эти сущности на примере простого приложения RAG, которое извлекает документы из индекса и генерирует ответ.

<img src="../images/primit.png" width="600" height="400" style="border-radius: 0.75rem;" alt="LangChain Image" />

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Чтобы начать пользоваться `LangSmith` достаточно:
* Перейти по [ссылке](https://smith.langchain.com) и зарегистрировать учётную запись
* Сгенерировать API-ключ
* Добавить пару строк в код, и смотреть за работой вашего сервиса в удобном вёб-интерфейсе.

На бесплатном тарифе доступно 5000 трейсов в месяц.<br>
    

In [2]:
# Этих строк уже будет достаточно, больше никакого дополнительного кода не нужно
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_API_KEY'] = getpass(prompt='Введите ваш LANGSMITH API ключ')

Введите ваш LANGSMITH API ключ ········


In [7]:
llm.invoke('Привет! Как дела?')

AIMessage(content='Привет! У меня всё хорошо, спасибо! Как дела у тебя?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 13, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': None, 'id': 'chatcmpl-BvnliXjuKZpIuYUmxPg3SUcUtQYSx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e1e88d4f-92c2-417a-be28-ad37b69122a5-0', usage_metadata={'input_tokens': 13, 'output_tokens': 15, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Посмотрим на наш первый трейс:
<img src="../images/trace1.png" width="600" height="400" style="border-radius: 0.75rem;" alt="LangChain Image" />

Видим, потраченные токены и деньги, составные части, запрос - ответ.

In [ ]:
# Теперь посмотрим на трэйс нашего агента
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "Сколько голов Месси и Криштиану Роналду забили в сумме?"})

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Здесь уже больше информации:
<img src="../images/trace2.png" width="600" height="400" style="border-radius: 0.75rem;" alt="LangChain Image" />

Слева в колонке видим отдельные шаги (`runs`) внутри трейса, можно на каждый нажать и посмотреть подробную информацию.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

В примере выше все трейсы со всех вызовов будут логироваться в системе, но это не всегда нужно и удобно. Чтобы определять какие именно трейсы будут отправляться в `LangSmith`, существуют 2 способа:
* Через `callback` и трейсер
* Через менеджер контекста

Посмотрим на прмерах.<br>
    

In [9]:
# Сначала стоит отключить логирование всех 
os.environ['LANGSMITH_TRACING'] = "false"

In [12]:
# Возьмём цепочку из прошлых уроков
from operator import itemgetter
from langchain_classic.prompts import ChatPromptTemplate
from langchain_classic.schema.output_parser import StrOutputParser


prompt1 = ChatPromptTemplate.from_template("В каком городе родился {person}?")
prompt2 = ChatPromptTemplate.from_template(
    "В какой стране находится город {city}? Ответь на {language} языке.")

chain1 = prompt1 | llm | StrOutputParser()

# Ключ 'city' получаем из chain1, а ключ 'language' достаём из запроса с помощью itemgetter
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | llm
    | StrOutputParser()
)

In [13]:
# Через callback
from langchain_classic.callbacks.tracers import LangChainTracer

tracer = LangChainTracer()
chain2.invoke({"person": "Колумб", "language": "испанском"}, config={"callbacks": [tracer]})

'Cristóbal Colón nació en Génova, que en ese momento era una república independiente y hoy forma parte de Italia.'

In [16]:
# Через менеджер контекста
from langchain_core.tracers.context import tracing_v2_enabled


with tracing_v2_enabled():
  res = chain2.invoke({"person": "Пушкин", "language": "немецкий"})
res

'Der Dichter Alexander Sergejewitsch Puschkin wurde in Russland geboren.'

In [17]:
# Без трейса
chain2.invoke({"person": "Месси", "language": "английский"})

'Lionel Messi was born in the city of Rosario, Argentina.'

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Видим, что в списке трейсов появились Колумб и Пушкин, но не появился Месси:
<img src="../images/trace3.png" width="600" height="400" style="border-radius: 0.75rem;" alt="LangChain Image" />

# <center id="c2"> 🔃🔪 Разделяем трейсы на проекты!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

По умолчанию все наши трейсы сыпятся в проект с именем `default`, и со временем будет трудно разобраться, где какие трейсы лежат и к чему относятся.

Разбивать трейсы на проекты (привязывать их к определенному проекту) можно 2-мя способами:
1. **Статически** - задать имя проекта в переменной среды, тогда все вызовы будут сыпаться в один проект. Делается командой `os.environ['LANGSMITH_PROJECT'] = "my-project"`
2. **Динамически** - при каждом вызове в конфиге указывать в какой проект отправлять трейс.

Разберём второй случай:

In [18]:
# Имя проекта можем указывать в трейсере

tracer = LangChainTracer(project_name="Chain Project")
chain2.invoke({"person": "Колумб", "language": "испанском"}, config={"callbacks": [tracer]})

'Cristóbal Colón nació en Génova, que en ese momento era una república independiente y ahora es parte de Italia.'

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Для второго проекта возьмём RAG-агента из предыдущего урока:

In [20]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_classic.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://allopizza.su/spb/kupchino/about")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

embeddings = OpenAIEmbeddings(api_key=course_api_key, model='text-embedding-3-small', 
                              base_url="https://api.vsellm.ru/")

db_embed = FAISS.from_documents(texts, embeddings)

retriever = db_embed.as_retriever()

tool = create_retriever_tool(
    retriever, # наш ретривер
    "search_web", # имя инструмента
    "Searches and returns data from page", # описание инструмента подается в ЛЛМ
)
tools = [tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [21]:
with tracing_v2_enabled(project_name="RAG Project"):
    result = agent_executor.invoke({"input": "Из скольки авто состоит автопарк компании?"})
result["output"]

'Автопарк компании "Алло! Пицца" состоит из более чем 200 легковых автомобилей.'

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    

Видим, что появились 2 новых проекта:
<img src="../images/trace_pr.png" width="600" height="400" style="border-radius: 0.75rem;" alt="LangChain Image" />

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
А вот так выглядит трейс агента с RAG:
<img src="../images/trace_rag.png" width="600" height="400" style="border-radius: 0.75rem;" alt="LangChain Image" />

# <center id="c2"> 🎭 Кастомизация трейсов!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Для упрощения поиска, фильтрации, наглядности можно кастомизировать трейсы, навешивая им:
* тэги
* метаданные
* уникальные имена и айдишники

Продемонстрируем всё на одном примере:

In [22]:
import uuid

my_uuid = uuid.uuid4()

config = {
    "callbacks": [tracer], 
    "tags": ["Chain invoke"], # указываем тэги
    "metadata": {"person": "Колумб", "language": "испанский"}, # указываем метаданные
    "run_name": "Test new data", # указываем имя для запуска
    "run_id": my_uuid, # указываем id
}


tracer = LangChainTracer(project_name="Chain Project")
chain2.invoke({"person": "Колумб", "language": "испанский"}, config=config)

'Cristóbal Colón nació en la ciudad de Génova, que se encuentra en el territorio de la actual Italia.'

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Посмотрим как изменился трейс после кастомизации:
<img src="../images/trace_cust.png" width="600" height="400" style="border-radius: 0.75rem;" alt="LangChain Image" />

# <center id="part6"> 🧸 Выводы и заключения ✅

<div class="alert alert-info">

В этом уроке рассмотрели инструменты для отладки приложений на базе LLM:
* Debug режим, встроенный в `LangChain` - подходит для отладки небольших Runnables и запросов, не очень наглядно и удобно.
* Специальный сервис для отслеживания трейсов `LangSmith`, который заводится с 2-х строк кода и позволяет отлаживать работу сервиса в удобном веб-интерфейсе.

- мы рассмотрели лишь часть возможностей `LangSmith`и познакомились с понятием трейса. Также платформа позволяет отслеживать качество ответов сервиса на собственных бенчмарках и отлаживать промпты.
- для использования `LangSmith` не обязательно использовать `LangChain`.

### Из минусов:
- `LangSmith` платный и закрытый
- Данные уходят за контур вашего сервиса, что не всегда может устроить заказчиков
- Недоступно при отсутствии интернет подключения (но, можно развернуть локально)


<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
#### Альтернативы:
Существуют альтернативные и опенсорсные решения, которые можно развернуть локально и хранить трейсы у себя. 

У этих библиотек очень похожий интерфейс и функционал с `LangSmith`:
* [LangFuse](https://langfuse.com) - open-source платформа, но так же может по подписке сервить ваши трейсы (есть бесплатный тариф)
* [Phoenix](https://github.com/Arize-ai/phoenix) - open-source библиотека от `Azure AI`, разворачивается локально, поднимает локальный веб-сервер, так же в пару строк подключается к `LangChain`, все трейсы хранятся у вас и отображаются в приятном веб-интерфейсе.